In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn import *
from torch.optim import *
from torchvision.models import *
from sklearn.model_selection import *
from sklearn.metrics import *
import wandb
import nltk
from nltk.stem.porter import *
PROJECT_NAME = "Natural-Language-Processing-with-Disaster-Tweets"
np.random.seed(55)
stemmer = PorterStemmer()
device = 'cpu'

In [7]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
tokenize("Testing this tokenize function")

['testing', 'this', 'tokenize', 'function']

In [11]:
def stem(word):
    return stemmer.stem(word.lower())

In [12]:
stem('organic')

'organ'

In [13]:
def words_to_int(words,all_words):
    new_words = []
    for word in words:
        new_words.append(stem(word))
    list_of_os = np.zeros(len(all_words))
    for i in range(len(all_words)):
        if all_words[i] in new_words:
            list_of_os[i] = 1.0
    return list_of_os

In [14]:
words_to_int(["test"],["testing","I","test","grswgre"])

array([0., 0., 1., 0.])

In [15]:
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [16]:
data = pd.read_csv('/content/drive/MyDrive/train.csv')
data = data.sample(frac=1)
data = data.sample(frac=1)
data = data.sample(frac=1)

In [17]:
X = data['text']
y = data['target']

In [18]:
all_words = []
tags = []

In [19]:
from tqdm import tqdm

In [20]:
for x_iter,y_iter in tqdm(zip(X,y)):
    x_iter = tokenize(x_iter)
    new_x_iter = []
    for x_iter_i in x_iter:
        new_x_iter.append(stem(x_iter_i))
    all_words.extend(new_x_iter)
    tags.append(y_iter)

7613it [00:04, 1681.84it/s]


In [21]:
np.random.shuffle(all_words)
np.random.shuffle(all_words)

In [22]:
all_words = sorted(set(all_words))

In [23]:
tags = sorted(set(tags))

In [24]:
new_X = []
new_y = []

In [25]:
for X_iter,y_iter in tqdm(zip(X,y)):
    new_X.append(words_to_int(X_iter,all_words))
    new_y.append(tags.index(y_iter))

7613it [04:24, 28.83it/s]


In [26]:
X = np.array(new_X)
y = np.array(new_y)

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.0625,shuffle=True)

In [28]:
X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_test = torch.from_numpy(y_test).to(device)

In [29]:
class Model(Module):
    def __init__(self,activation=ReLU,neurons=512):
        super().__init__()
        self.activation = activation()
        self.output_activation = Sigmoid()
        self.linear1 = Linear(len(all_words),neurons)
        self.linear2 = Linear(neurons,neurons*2)
        self.linearbn = BatchNorm1d(neurons*2)
        self.linear3 = Linear(neurons*2,neurons*2)
        self.linear4 = Linear(neurons*2,neurons*3)
        self.linear5 = Linear(neurons*3,neurons*2)
        self.output = Linear(neurons*2,1)
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linearbn(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output_activation(self.output(preds))
        return preds

In [30]:
model = Model(activation=LeakyReLU,neurons=256).to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [31]:
def accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X.float())
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(y_batch)
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [32]:
def g_loss(model,X,y):
    preds = model(X.float())
    loss = criterion(preds.view(-1),y.float().view(-1))
    return loss.item()

In [33]:
iter_epochs = tqdm(range(epochs))
for _ in iter_epochs:
    for i in range(0,len(X_train),batch_size):
        try:
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            preds = model(X_batch.float())
            loss = criterion(preds.float().view(-1),y_batch.float().view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        except:
            pass
    model.eval()
    print(f"Val Accuracy = {accuracy(model,X_test,y_test)} | Accuracy = {accuracy(model,X_train,y_train)} | Val Loss = {g_loss(model,X_test,y_test)} | Loss = {g_loss(model,X_train,y_train)}")
    model.train()

  1%|          | 1/100 [00:28<47:11, 28.60s/it]

Val Accuracy = 66.8 | Accuracy = 66.4 | Val Loss = 0.22018006443977356 | Loss = 0.2166014164686203


  2%|▏         | 2/100 [01:01<51:09, 31.32s/it]

Val Accuracy = 64.5 | Accuracy = 65.2 | Val Loss = 0.21716555953025818 | Loss = 0.21677939593791962


  3%|▎         | 3/100 [01:32<50:23, 31.17s/it]

...

Val Accuracy = 58.4 | Accuracy = 81.6 | Val Loss = 0.36377212405204773 | Loss = 0.1488240361213684


 99%|█████████▉| 99/100 [53:26<00:33, 33.18s/it]

Val Accuracy = 60.699999999999996 | Accuracy = 82.6 | Val Loss = 0.35354921221733093 | Loss = 0.1398986130952835


100%|██████████| 100/100 [53:59<00:00, 32.39s/it]

Val Accuracy = 61.1 | Accuracy = 83.0 | Val Loss = 0.33804282546043396 | Loss = 0.13835644721984863


In [34]:
model.eval()
preds = model(X_test.float())

In [35]:
new_test = []

In [36]:
for X_iter in tqdm(test['text']):
    new_test.append(words_to_int(X_iter,all_words))

100%|██████████| 3263/3263 [01:55<00:00, 28.28it/s]


In [37]:
new_test = torch.from_numpy(np.array(new_test)).to(device)

In [38]:
preds = model(new_test.float())

In [39]:
ids = test['id']

In [40]:
submission = {
    "id":[],
    "target":[]
}

In [41]:
for pred,id in tqdm(zip(preds,ids)):
    submission['id'].append(id)
    submission['target'].append(int(torch.round(pred)))

3263it [00:00, 101418.45it/s]


In [42]:
submission = pd.DataFrame(submission)

In [43]:
submission.to_csv("./base.csv",index=False)